# Skapa en modell 
som ger underlag för att köpa eller sälja kryptovaluta

In [1]:
import pandas as pd
import numpy as np
import binance
import momentum as mom

## Skapa train/test data

In [5]:
### Kör denna kod för att skapa en dataframe med train/test data ###
###               Gör interupt när tillräckligt finns            ###
# set timer
import time
start_time = time.time()
mom.start()
duration = time.time() - start_time

[24840.55       24830.382      24833.84208    24819.38707394]
[24839.77       24832.844      24833.84208    24818.32522848]
[24839.6        24835.244      24833.84208    24817.96795437]


In [11]:
duration = time.time() - start_time
# format duration to minutes and seconds
duration = divmod(duration, 60)
str_duration = str(duration[0]) + ' minutes and ' + str(int(duration[1])) + ' seconds'
print(str_duration )
print(mom.df.tail(10))
mom.df.to_csv('data.csv')

6.0 21
6.0 minutes and 21 seconds
                             pris      sma_5      sma_20        fib38  \
2023-02-20 19:01:12.212  24827.98  24826.986  24824.7875  24825.83316   
2023-02-20 19:01:13.213  24826.59  24826.996  24824.9275  24825.83316   
2023-02-20 19:01:14.211  24827.27  24827.010  24825.0820  24825.83316   
2023-02-20 19:01:15.211  24827.46  24827.286  24825.2050  24825.83316   
2023-02-20 19:01:16.211  24827.60  24827.380  24825.3500  24825.83316   
2023-02-20 19:01:17.213  24828.09  24827.402  24825.6365  24825.90114   
2023-02-20 19:01:18.210  24828.21  24827.726  24825.8750  24826.21596   
2023-02-20 19:03:32.230  24840.55  24830.382  24826.7390  24833.84208   
2023-02-20 19:03:33.229  24839.77  24832.844  24827.5460  24833.84208   
2023-02-20 19:03:34.231  24839.60  24835.244  24828.3765  24833.84208   

                             fib50        fib62    boll_sma  boll_std  \
2023-02-20 19:01:12.212  24825.170  24824.50684  24824.7875  1.602563   
2023-02-20 19:01

### y-värdena: en plus två tidseneheter framåt i df

In [58]:
df=mom.df.copy()
# df.index

In [62]:
df['m_strategy'] = ['hold'] * len(df)
for i,val in enumerate(df):
    print(df.iloc[i]['m_strategy'])
    ix = df.index[i]
    df.loc[ix,'m_strategy'] = 'sell'
    # df['m_strategy'].iloc[i] = 'buy'
    print(df.loc[ix])
    break

hold
pris          24818.75
sma_5              NaN
sma_20             NaN
fib38              NaN
fib50              NaN
fib62              NaN
boll_sma           NaN
boll_std           NaN
upper_band         NaN
lower_band         NaN
b_strategy        hold
m_strategy        sell
Name: 2023-02-20 18:53:55.147000, dtype: object


In [ ]:
def get_b_strategy(row):
    i = row.name
    print(i)
    if df['pris'].shift(-1).loc[i] < df['upper_band'].shift(-1).loc[i] and df['pris'].shift(0).loc[i] > df['upper_band'].shift(0).loc[i]:
        return 'sell'
    elif df['pris'].shift(-1).loc[i] > df['lower_band'].shift(-1).loc[i] and df['pris'].shift(0).loc[i] < df['lower_band'].shift(0).loc[i]:
        return 'buy'
    else:
        return 'hold'


    
df['b_strategy'] = df.apply(get_b_strategy, axis=1)

df['b_strategy'].tail(2)
print(df['pris'].shift(1).iloc[-1])
df

In [ ]:
client = binance.Client()
r = client.get_historical_klines('ETHBTC', client.KLINE_INTERVAL_1MINUTE, '17-Jan-2023', '20-Jan-2023')
df = pd.DataFrame(r,columns=['datum', 'open','low','high','close','volume','close_time','quote_asset_volume','number_of_trades','taker_buy_base_asset_volume','taker_buy_quote_asset_volume','ignore'])
df = df[['datum','open','high','low','close','volume']]
# OHLCV values (Open time, Open, High, Low, Close, Volume, Close time, Quote asset volume, 
# Number of trades, Taker buy base asset volume, Taker buy quote asset volume, Ignore)
df.datum = pd.to_datetime(df.datum, unit='ms')
df.set_index('datum', inplace=True)

In [ ]:
df